In [1]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy.io import savemat

In [2]:
# Change fonts and specify font size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
FONT_SIZE = 12

In [3]:
# Define necessary functions
# Function to save files
def save_file(data, file_name, file_directory):
    
    # Save the file as a .mat file
    savemat(os.path.join(file_directory, file_name), {'data': data})

    print(f'Saved {file_name} to {file_directory}')

In [4]:
# Define function to renormalize the dataset
def renormalize_data(data_to_be_renormalized, training_mean, training_range):

    renormalized_data = (data_to_be_renormalized * training_range) + training_mean

    return renormalized_data

In [5]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized test data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'training_results')

# Define directory for the predicted results
predicted_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'prediction_results')

In [6]:
# Load the normalized test subsets for displacement data
print('Loading the normalized test subsets for displacement data...')
normalized_test_displacement_data = np.load(os.path.join(normalized_data_directory, 'normalized_test_displacement_data.npy'))

Loading the normalized test subsets for displacement data...


In [7]:
# Load the normalized test subsets for force data
print('Loading the normalized test subsets for force data...')
normalized_test_force_data = np.load(os.path.join(normalized_data_directory, 'normalized_test_force_data.npy'))

Loading the normalized test subsets for force data...


In [8]:
# Print the shapes of the displacement and force data
print(f'The shape of displacement data is {normalized_test_displacement_data.shape[1:]}.')
print(f'The shape of force data is {normalized_test_force_data.shape[1:]}.')

The shape of displacement data is (251, 74).
The shape of force data is (251, 894).


In [9]:
# Load the best trained model
print('Loading the best trained model...')
best_trained_model = tf.keras.models.load_model(os.path.join(trained_results_directory, 'best_model.h5'), compile = False)

Loading the best trained model...


2024-02-06 16:32:59.163805: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-02-06 16:32:59.163830: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: swimlab-linux
2024-02-06 16:32:59.163837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: swimlab-linux
2024-02-06 16:32:59.163924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.154.5
2024-02-06 16:32:59.163949: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.146.2
2024-02-06 16:32:59.163956: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 535.146.2 does not match DSO version 535.154.5 -- cannot find working devices in this configuration
2024-02-06 16:32:59.164316: I tensorflow/core/platform/cpu_feature_guard.cc:151] T

In [10]:
# Print model summary
print(best_trained_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 251, 74)]         0         
                                                                 
 conv1d (Conv1D)             (None, 251, 600)          4218600   
                                                                 
 batch_normalization (BatchN  (None, 251, 600)         2400      
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 251, 400)          22800400  
                                                                 
 batch_normalization_1 (Batc  (None, 251, 400)         1600      
 hNormalization)                                                 
                                                                 
 conv1d_2 (Conv1D)           (None, 251, 100)          380010

In [11]:
# Make predictions using the best trained model
print('Making predictions using the best trained model...')
predicted_normalized_test_force_data = best_trained_model.predict(normalized_test_displacement_data)

Making predictions using the best trained model...


In [12]:
# Load the training mean and range
normalizing_force_parameters = np.load(os.path.join(normalized_data_directory, 'normalizing_force_parameters.npy'), allow_pickle=True)
force_mean = normalizing_force_parameters.item().get('force_mean')
force_range = normalizing_force_parameters.item().get('force_range')

In [13]:
# Renormalize the target and force data
print('Renormalizing the target and force data...')
predicted_test_force_data = renormalize_data(predicted_normalized_test_force_data, force_mean, force_range)
target_test_force_data = renormalize_data(normalized_test_force_data, force_mean, force_range)

Renormalizing the target and force data...


In [14]:
# Calculate the error
numerator = np.sum(np.sum((np.abs(target_test_force_data - predicted_test_force_data))**2, axis=1), axis=1)
denominator = np.sum(np.sum((np.abs(target_test_force_data))**2, axis=1), axis=1)
error = numerator/denominator*100

In [15]:
# Analyze the error values
# Sort the error values for Q1, Median, and Q3
sorted_error = np.sort(error)

In [16]:
# Minimum error
min_error_location = np.where(error == sorted_error[0])
min_error = sorted_error[0]

In [17]:
# Q1 error
q1_location = int(len(sorted_error)/4)
q1_error = sorted_error[q1_location]

In [18]:
# Median error
median_location = int(len(sorted_error)/2)
median_error = sorted_error[median_location]

In [19]:
# Q3 error
q3_location = int(3*len(sorted_error)/4)
q3_error = sorted_error[q3_location]

In [20]:
# Max error
max_error_location = np.where(error == sorted_error[-1])
max_error = sorted_error[-1]

In [21]:
# Print the summary of the error
print(f'The minimum error is {min_error:.2f}%.')
print(f'The Q1 error is {q1_error:.2f}%.')
print(f'The median error is {median_error:.2f}%.')
print(f'The Q3 error is {q3_error:.2f}%.')
print(f'The maximum error is {max_error:.2f}%.')
print(f'The mean error is {np.mean(error):.2f}%.')

The minimum error is 0.10%.
The Q1 error is 0.36%.
The median error is 0.55%.
The Q3 error is 0.88%.
The maximum error is 5.40%.
The mean error is 0.72%.


In [22]:
# Save the target and predicted force data
print('Saving the target and predicted force data...')
save_file(predicted_test_force_data, 'predicted_test_force_data.mat', predicted_results_directory)
save_file(target_test_force_data, 'target_test_force_data.mat', predicted_results_directory)

Saving the target and predicted force data...
Saved predicted_test_force_data.mat to /media/swimlab/8e0a5339-75ae-4b57-aaae-375e5bb09ac3/ann-drm-vector-wave/scripts/cnn/../../cnn/prediction_results
Saved target_test_force_data.mat to /media/swimlab/8e0a5339-75ae-4b57-aaae-375e5bb09ac3/ann-drm-vector-wave/scripts/cnn/../../cnn/prediction_results
